# Pixel level rnn on MNIST
This is a simple exercise to use RNN model for mnist classification. The model scan MNIST images pixel by pixel to classify the digits.

In [5]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials import mnist

## data provider

In [6]:
class mnist_provider(object):
  """MNIST data provider."""
  def __init__(self, data_directory, split="train"):
    mnist_data = mnist.input_data.read_data_sets(data_directory, one_hot=True)
    if split == "train":
      self.mnist_data = mnist_data.train
    elif split == "valid":
      self.mnist_data = mnist_data.validation
    elif split == "test":
      self.mnist_data = mnist_data.test

  def next_batch(self, batch_size):
    images, one_hot_labels = self.mnist_data.next_batch(batch_size)
    images = np.reshape(images, [-1, 28, 28, 1], order='C')
    return images, one_hot_labels

In [7]:
data_dir = "/Users/gamal/git_local_repo/playground/data/mnist"
data_provider_train = mnist_provider(data_dir, split='train')
data_provider_valid = mnist_provider(data_dir, split='valid')
data_provider_test = mnist_provider(data_dir, split='test')

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /Users/gamal/git_local_repo/playground/data/mnist/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /Users/gamal/git_local_repo/playground/data/mnist/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /Users/gamal/git_local_repo/playground/data/mnist/t10k-images-idx3-ubyte.gz
Extracting /Users/gamal/git_local_repo/playground/data/mnist/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Extracting /Users/gamal/git_local_repo/playground/data/mnist/train-images-idx3-ubyte.gz
Extracting /Users/gamal/git_local_repo/playground/data/mn

In [18]:
tf.reset_default_graph()
g = tf.Graph()
in_dim = 1  # input dimensionality is 1 (1 pixel at a time)
T = 28*28  # times is the total number of pixels

with g.as_default():
  x = tf.placeholder(dtype=tf.float32, shape=(None, T, in_dim))
  labels = tf.placeholder(dtype=tf.float32, shape=(None))
  
  # convert in put to a 
  x_t = tf.unstack(x, T, 1)

In [19]:
x_t

[<tf.Tensor 'unstack:0' shape=(?, 1) dtype=float32>,
 <tf.Tensor 'unstack:1' shape=(?, 1) dtype=float32>,
 <tf.Tensor 'unstack:2' shape=(?, 1) dtype=float32>,
 <tf.Tensor 'unstack:3' shape=(?, 1) dtype=float32>,
 <tf.Tensor 'unstack:4' shape=(?, 1) dtype=float32>,
 <tf.Tensor 'unstack:5' shape=(?, 1) dtype=float32>,
 <tf.Tensor 'unstack:6' shape=(?, 1) dtype=float32>,
 <tf.Tensor 'unstack:7' shape=(?, 1) dtype=float32>,
 <tf.Tensor 'unstack:8' shape=(?, 1) dtype=float32>,
 <tf.Tensor 'unstack:9' shape=(?, 1) dtype=float32>,
 <tf.Tensor 'unstack:10' shape=(?, 1) dtype=float32>,
 <tf.Tensor 'unstack:11' shape=(?, 1) dtype=float32>,
 <tf.Tensor 'unstack:12' shape=(?, 1) dtype=float32>,
 <tf.Tensor 'unstack:13' shape=(?, 1) dtype=float32>,
 <tf.Tensor 'unstack:14' shape=(?, 1) dtype=float32>,
 <tf.Tensor 'unstack:15' shape=(?, 1) dtype=float32>,
 <tf.Tensor 'unstack:16' shape=(?, 1) dtype=float32>,
 <tf.Tensor 'unstack:17' shape=(?, 1) dtype=float32>,
 <tf.Tensor 'unstack:18' shape=(?, 1) 